In [1]:
# 大約要跑 15 分鐘
from pyspark import SparkConf, SparkContext
import math, sys, glob, random
import sympy
import numpy as np
from nltk import ngrams, word_tokenize
import json
import matplotlib.pyplot as plt
import os
from collections import defaultdict
from itertools import *
import csv

# from tqdm.notebook import trange, tqdm
plt.style.use('ggplot')
conf=SparkConf().set("spark.default.parallelism", 4)
# conf = SparkConf().setMaster("local").setAppName("Final")
sc = SparkContext(conf=conf)

In [2]:
%%time
with open('data_path.json', encoding='utf-8') as f:
    data_path=json.load(f)

CPU times: user 180 ms, sys: 72 ms, total: 252 ms
Wall time: 252 ms


In [3]:
data_path[:10]

['./data/all/politics/politics/20140225/M.1393291310.A.21C.json',
 './data/all/politics/politics/20140225/M.1393281343.A.02B.json',
 './data/all/politics/politics/20111121/M.1321861642.A.636.json',
 './data/all/politics/politics/20111121/M.1321875121.A.0FE.json',
 './data/all/politics/politics/20131110/M.1384016795.A.445.json',
 './data/all/politics/politics/20131110/M.1384018254.A.402.json',
 './data/all/politics/politics/20131110/M.1384094585.A.376.json',
 './data/all/politics/politics/20090827/M.1251349280.A.6D8.json',
 './data/all/politics/politics/20090827/M.1251380656.A.33E.json',
 './data/all/politics/politics/20090827/M.1251356301.A.153.json']

In [4]:
data = sc.parallelize(data_path)

In [5]:
%%time
# 從路徑轉換成 instance
def load_instance(x):
    with open(x) as f:
        d = json.load(f)
    return d

data = data.map(load_instance)
load_instance(data_path[0])

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 5.25 ms


{'article_id': 'M.1393291310.A.21C',
 'article_title': '[討論] 柯醫師該做的事',
 'author': 'kevinjl (無神論者-無神得自由)',
 'board': 'politics',
 'content': '1.堅持言論自由，保障言論自由 以上是廢話 他該做的事情是：宣傳在台灣任何個人有主張 與中國統一是對台灣有利的事情。 柯醫師不必同意與中國統一是對台灣有利的事情， 但是他必須宣傳，台灣人有權利主張與中國統一是對台灣有利的事情。 他需要利用上媒體曝光的機會，宣傳這種言論自由的理念。 這是目前綠營作不到，而他可以做的事情。 2. 保障目前的既有利益，宣傳公平正義是提高弱勢者的利益 而非砍除既有利益者的利益來實踐。 如果柯醫師能夠在日後的宣傳，言行政策上 讓民眾充分感受到這點，那麼他就會當選了 反之，多數選民還是會選擇讓自己有利益的權貴。 ',
 'date': 'Tue Feb 25 09:21:47 2014',
 'ip': 'None',
 'message_conut': {'all': 5, 'boo': 0, 'count': 1, 'neutral': 4, 'push': 1},
 'messages': [{'push_content': '柯文哲本來就不排斥統一，臺獨不是他的主張',
   'push_ipdatetime': '183.5.28.167 02/26 10:35',
   'push_tag': '推',
   'push_userid': 'lkcs'},
  {'push_content': '這才是他不會加入民進黨的真正原因，臺獨黨綱',
   'push_ipdatetime': '183.5.28.167 02/26 10:35',
   'push_tag': '→',
   'push_userid': 'lkcs'},
  {'push_content': '但柯文哲也不是無條件統一，是有條件的',
   'push_ipdatetime': '183.5.28.167 02/26 10:36',
   'push_tag': '→',
   'push_userid': 'lkcs'},
  {'push_content': '民主自由人權

In [6]:
len(data_path)

865042

In [7]:
data.take(1)

[{'article_id': 'M.1393291310.A.21C',
  'article_title': '[討論] 柯醫師該做的事',
  'author': 'kevinjl (無神論者-無神得自由)',
  'board': 'politics',
  'content': '1.堅持言論自由，保障言論自由 以上是廢話 他該做的事情是：宣傳在台灣任何個人有主張 與中國統一是對台灣有利的事情。 柯醫師不必同意與中國統一是對台灣有利的事情， 但是他必須宣傳，台灣人有權利主張與中國統一是對台灣有利的事情。 他需要利用上媒體曝光的機會，宣傳這種言論自由的理念。 這是目前綠營作不到，而他可以做的事情。 2. 保障目前的既有利益，宣傳公平正義是提高弱勢者的利益 而非砍除既有利益者的利益來實踐。 如果柯醫師能夠在日後的宣傳，言行政策上 讓民眾充分感受到這點，那麼他就會當選了 反之，多數選民還是會選擇讓自己有利益的權貴。 ',
  'date': 'Tue Feb 25 09:21:47 2014',
  'ip': 'None',
  'message_conut': {'all': 5, 'boo': 0, 'count': 1, 'neutral': 4, 'push': 1},
  'messages': [{'push_content': '柯文哲本來就不排斥統一，臺獨不是他的主張',
    'push_ipdatetime': '183.5.28.167 02/26 10:35',
    'push_tag': '推',
    'push_userid': 'lkcs'},
   {'push_content': '這才是他不會加入民進黨的真正原因，臺獨黨綱',
    'push_ipdatetime': '183.5.28.167 02/26 10:35',
    'push_tag': '→',
    'push_userid': 'lkcs'},
   {'push_content': '但柯文哲也不是無條件統一，是有條件的',
    'push_ipdatetime': '183.5.28.167 02/26 10:36',
    'push_tag': '→',
    'push_userid': 'lkcs'},
   {'p

In [8]:
%%time
TAG_SCORE = {'推':1, '噓':-1, '→':0}
def count_user_interaction(x, tag):
    used = ['author', 'messages']
    for i in used:
        if i not in x or x[i] == None:
            return [[]]
    main_user = x['author'].split(' ')[0]
    user_comments = defaultdict(lambda : 0)
    
    # 計算分數
    for i in x['messages']:
        
        uid = i['push_userid']
        uscore = tag[i['push_tag']]
        user_comments[uid] += uscore
    
    # 建立 graph
    pairs = []
    for i in user_comments:
        is_neg = user_comments[i]
        score = user_comments[i]
        if score == 0:
            continue
        if is_neg:
            pair = [(i, main_user), score]
        else:
            pair = [(main_user, i), score]
        pairs.append(pair)
        
    return pairs

# count_user_interaction(load_instance(data_path[0]), TAG_SCORE)
data = data.flatMap(lambda x:count_user_interaction(x, TAG_SCORE)).filter(lambda x:x!=[])
data.take(5)

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 99.4 ms


[[('lkcs', 'kevinjl'), 1],
 [('mackywei', 'stevegreat08'), 1],
 [('goodcial', 'stevegreat08'), 4],
 [('yukiss', 'stevegreat08'), 1],
 [('starcloud', 'stevegreat08'), 1]]

In [9]:
%%time
data = data.groupByKey().mapValues(sum)
data = data.repartition(1).groupByKey().mapValues(sum).repartition(1)
data.take(5)



CPU times: user 64 ms, sys: 12 ms, total: 76 ms
Wall time: 10min 18s


[(('Bonjwa', 'oppo5566'), 1),
 (('vic9503', 'ssss0960193'), 0),
 (('qqwweerrqq', 'Tapqou'), 1),
 (('garyiopp15', 'asole'), 1),
 (('g1254501', 'psl7634'), 1)]

In [10]:
data.saveAsTextFile('u2u_interactive_graph')

In [11]:
# %%time
# graph = data.collect()

# graph[0]


In [12]:
# with open('graph.csv', 'w') as f:
#     json.dump(graph, f, indent=4, ensure_ascii=False)
# graph[0]

In [13]:
############################################################

In [14]:
# user to page
